In [3]:
pip install dash

  Created wheel for dash: filename=dash-1.11.0-cp37-none-any.whl size=72530 sha256=3f49ddbcb9ceb97adf32d52d0abc635927473f97ed9f79b168cb87b3e318169a
  Stored in directory: C:\Users\maria\AppData\Local\pip\Cache\wheels\e8\29\9b\d4189bff04708f6fede29abd4fa390390734605854b5bff51b
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp37-none-any.whl size=5278 sha256=d6e4c765b0a0b99559b10a70d1b1435d392e96087137ce26047149bb1363e25c
  Stored in directory: C:\Users\maria\AppData\Local\pip\Cache\wheels\f7\e9\e4\5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
  Created wheel for dash-renderer: filename=dash_renderer-1.4.0-cp37-none-any.whl size=1164174 sha256=e7cc73eed97c8df5249e1d30b135b91813057c66eef0a012bc55226210723c4a
  Stored in directory: C:\Users\maria\AppData\Local\pip\Cache\wheels\bd\3f\ac\96265035b8ff03f50b14ca0fb75d3fb86320d202270507d9f1
  Created wheel for dash-core-components: filename=dash_core_components-1.9.1-cp37-none-any.whl size=3407360 sha256=3055d7aa10d6aa4c

In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('../Resources/airdata.csv')

available_indicators = df['City'].unique()



In [7]:
app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='max'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='max'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Wuhan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-month--slider',
        min=df['Date'].min(),
        max=df['Date'].max(),
        value=df['Date'].max(),
        marks={str(month): str(month) for month in df['Date'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])

SyntaxError: unexpected EOF while parsing (<ipython-input-7-39326f6295c3>, line 65)

In [8]:


def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 month_value):
    dff = df[df['Date'] == month_value]

    return {
        'data': [dict(
            x=dff[dff['City'] == xaxis_column_name]['max'],
            y=dff[dff['City'] == yaxis_column_name]['max'],
            text=dff[dff['City'] == yaxis_column_name]['max'],
            customdata=dff[dff['City'] == yaxis_column_name]['max'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }
def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Month'],
            y=dff['max'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }


@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])


SyntaxError: unexpected EOF while parsing (<ipython-input-8-adf2cc121064>, line 59)

In [9]:
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    city_name = hoverData['points'][0]['customdata']
    dff = df[df['City'] == city_name]
    dff = dff[dff['City'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(city_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['max'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['City'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)



In [10]:
if __name__ == '__main__':
    app.run_server(debug=True)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 830-646-778
Debugger PIN: 830-646-778
Debugger PIN: 830-646-778
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\Users\maria\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [11]:
%tb

SystemExit: 1